In [1]:
import pandas as pd
import numpy as np
import pymarc
import re
import csv


In [15]:
path= 'data/hertz.mrc'
out_file_path = 'data/csv_files/hertz.csv'

COLUMNS = ['id', 'signature', 'inventory_nr']

In [26]:
with open(path, 'rb') as fh:
    reader = pymarc.MARCReader(fh, to_unicode=True, force_utf8=True, utf8_handling='ignore')

    with open(out_file_path, 'w', encoding='utf8') as out:

         #Dictwriter for better performance, write header

            writer = csv.DictWriter(out, fieldnames=COLUMNS)
            writer.writeheader()

            rows = []

            for record in reader:
                
                id = record['001']
                signature = record ['LOD' ]['3'] if record ['LOD' ] != None else None
                inventory_nr = record ['LOD' ]['6'] if record ['LOD' ] != None else None

                rows += [{
                    'id': id.value(), 
                    'signature': signature, 
                    'inventory_nr': inventory_nr
                }]

                #print(id.value())
                #print(record)


            writer.writerows(rows)


            

In [19]:
import requests
from bs4 import BeautifulSoup

requests.packages.urllib3.disable_warnings()


# Make a request to the website and parse the HTML
url = 'https://dlib.biblhertz.it/service/aufstellung'
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.content.decode('utf-8', 'ignore'), 'html.parser')


# create an empty dictionary to store the results
result_dict = {}


# Level 1 ----------------------------------------------------------------
#hoping that these are correct

level1_elements = soup.find_all(class_="level1")
for element in level1_elements:

    a = element.find('a')
    span = a.find('span')

    level1_sym = span.text.strip()
    level1_text = a.text.replace(level1_sym, '').strip()
   
    # print(level1_sym)

    # Level 2 ---------------------------------------------------------------
    level2_ul = element.find('ul', {'class': 'tree'})

    # create a dictionary to store the text and subfields of each element
    level2_dict = {}
     
    if level2_ul:
        level2_li = level2_ul.find_all('li')
        for li2 in level2_li:

            a = li2.find('a')
            span = a.find('span')

            level2_sym = span.text.strip()
            level2_text = a.text.replace(level2_sym, '').strip()
                
            #print('2:' + level2_sym +'/' + level2_text)

            # Level 3 --------------------------------------------------------

            level3_dict = {}

            level3_ul = li2.find('ul', {'class': 'tree'})

            if level3_ul:
                level3_li = level3_ul.find_all('li')

                for li3 in level3_li:
                
                    a = li3.find('a')
                    span = a.find('span')

                    level3_sym = span.text.strip()
                    level3_text = a.text.replace(level3_sym, '').strip()

                    if(level3_sym == ''):
                    
                        print('3:' + level3_sym + '/' + level3_text)
                
                    # Level 4 which is actually also level 3? -------------------------
                    
                    level4_ul = li3.find('ul', {'class': 'tree'})

                    if level4_ul:
                        level4_li = level4_ul.find_all('li')

                        for li4 in level4_li:

                            a = li4.find('a')
                            span = a.find('span')

                            level4_sym = span.text.strip()
                            level4_text = a.text.replace(level4_sym, '').strip()

                            print('4:' + level3_text + ', 4: ' + level4_text)



            # add the level  2 element 
            level2_dict[level2_sym] = {'text': level2_text, 'subfields': level3_dict}


# add level 1 element to result dict
result_dict[level1_sym] = {'text': level1_text, 'subfields': level2_dict}

                            
                            



3:/Lexika
3:/Nationale Allgemeinbibliographien
3:/System und Geschichte der Kunstwissenschaft (einschließlich Kunstkritik)
3:/Bibliographien
3:/Politische Geschichte allgemein
3:/einzelne Guiden
3:/Künstlergeschichte (nicht einzelne Künstler)
3:/Geschichte der Architektur (außer Profanbau speziell)
3:/Anthologien
3:/Quellenkundliche Literatur
4:Quellenkundliche Literatur, 4: zeitlich durchgehend
4:Quellenkundliche Literatur, 4: Kunst allgemein und Ästhetik, international bzw. mehrere Länder
4:Quellenkundliche Literatur, 4: idem, Italien
4:Quellenkundliche Literatur, 4: idem, sonstige einzelne Länder
4:Quellenkundliche Literatur, 4: Architektur, international bzw. mehrere Länder
4:Quellenkundliche Literatur, 4: idem, Italien
4:Quellenkundliche Literatur, 4: idem, sonstige einzelne Länder
4:Quellenkundliche Literatur, 4: Plastik, international bzw. mehrere Länder
4:Quellenkundliche Literatur, 4: idem, Italien
4:Quellenkundliche Literatur, 4: idem, sonstige einzelne Länder
4:Quellenkundli

In [30]:
df = pd.read_csv('data/bhr1.csv', error_bad_lines=False, sep=';')

C:\Users\hanna\AppData\Local\Temp/ipykernel_11848/1291591522.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('data/bhr1.csv', error_bad_lines=False, sep=';')


In [31]:
df

,lev,sys,numbis,text,vw
0,1,S xxx,yyy,...Bibliotheca Hertziana,NaN
1,1,S xxx,yyy,...Systematischer Standortkatalog,NaN
2,1,A,NaN,...Handbücherei,NaN
3,2,Aa,NaN,......Allgemeine Nachschlagewerke,NaN
4,3,Aa xxx,yyy,.........Lexika,NaN
...,...,...,...,...,...
5993,5,Ne 1036,NaN,...............idem (= interkontinental und Eu...,NaN
5994,5,Ne 1050,NaN,"...............Deutschland, Mittelalter",NaN
5995,5,Ne 1054,1066,"...............idem (= Deutschland), nach Jahr...",NaN
5996,5,Ne 1080,NaN,...............Österreich,NaN


In [60]:
level2 = df[df.lev == 2]

In [63]:
level1 = df[df.lev == 1]

In [64]:
level1

,lev,sys,numbis,text,vw
0,1,S xxx,yyy,...Bibliotheca Hertziana,NaN
1,1,S xxx,yyy,...Systematischer Standortkatalog,NaN
2,1,A,NaN,...Handbücherei,NaN
689,1,B,NaN,...Italienische Kunst,NaN
3022,1,D,NaN,...Topographie Rom,NaN
4236,1,F,NaN,...Reiseberichte,NaN
4297,1,G,NaN,...Quellenschriften und Quellenkunde,NaN
4490,1,H,NaN,...Ikonographie,NaN
5663,1,J,NaN,...Ornament,NaN
5689,1,M,NaN,...Kunst allgemein (ohne Italien speziell),NaN


In [61]:
level2

,lev,sys,numbis,text,vw
3,2,Aa,NaN,......Allgemeine Nachschlagewerke,NaN
75,2,Ab,NaN,......Kunstwissenschaft,NaN
261,2,Ad,NaN,......Ur- und Frühgeschichte,NaN
267,2,Ae,NaN,......Archäologie,NaN
291,2,Af,NaN,......Historische Hilfswissenschaften,NaN
324,2,Ag,NaN,"......Buch-, Bibliotheks- und Archivwesen",NaN
354,2,Ah,NaN,"......Geographie, Geologie, Astronomie",NaN
406,2,Ai,NaN,......Politische Geschichte,NaN
426,2,Ak,NaN,......Kirchengeschichte,NaN
438,2,Al,NaN,"......Kultur-, Sozial- und Sittengeschichte, V...",NaN
